In [1]:
import numpy as np
import pandas as pd
client = pd.read_csv('D://BA//test DA//New folder (2)//Book3.csv')
payment=pd.read_csv('D://BA//test DA//New folder (2)//Book4.csv')


In [3]:
import pandasql as psql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q)

In [26]:
#identify clientid have duplicated value
duplicate_rows = client[client.duplicated('client_id')]
print(duplicate_rows)

      client_id                 entity_type  entity_year_established
306         591  Australian Private Company                     2007
350         473      Individual/Sole Trader                     2008
816         591  Australian Private Company                     2015
1115        165      Individual/Sole Trader                     2006
1182       1262  Australian Private Company                     2016
1275        797  Australian Private Company                     2002


In [27]:
#decided to drop duplicate value because  client id is unique value and Each line represents a client
client.drop_duplicates('client_id',keep='last')

,client_id,entity_type,entity_year_established
0,786,Australian Private Company,2002
1,230,Australian Private Company,2008
2,282,Individual/Sole Trader,2001
3,447,Australian Private Company,2013
4,310,Individual/Sole Trader,2015
...,...,...,...
1282,815,Australian Private Company,2012
1283,201,Australian Private Company,2009
1284,492,Australian Private Company,2007
1285,765,Australian Private Company,2014


In [8]:
payment

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code
0,20175,927,1,1527012511,66.66,PAYMENT
1,8485,927,1,1511716095,66.66,PAYMENT
2,13778,927,1,1519319303,66.66,PAYMENT
3,22768,927,1,1529863724,66.66,PAYMENT
4,15698,927,1,1521738504,66.66,PAYMENT
...,...,...,...,...,...,...
25554,25075,1603,1280,1532023764,1666.68,PAYMENT
25555,24711,1603,1280,1531764560,0.01,PAYMENT
25556,25076,1603,1280,1532023764,64.99,PAYMENT
25557,25132,1627,1281,1532282886,0.01,PAYMENT


In [28]:

#Total payments for each client in 2018
totalpayment = """
    SELECT client.client_id,client.entity_type,SUM(p.payment_amt) AS total_payment 
    FROM client
    JOIN payment p ON client.client_id = p.client_id
    WHERE 
    strftime('%Y', datetime(p.transaction_date, 'unixepoch')) = '2018'
    AND p.payment_code <> 'DEFAULT'
    GROUP BY client.client_id,client.entity_type
"""

# Use the sqldf function from pandasql to execute the SQL query
totalpayment = sqldf(totalpayment, locals())

In [29]:
#rank within each entity type
rank = """
    SELECT client_id,entity_type,total_payment,
    DENSE_RANK() OVER (PARTITION BY entity_type ORDER BY total_payment DESC) AS rank
    FROM totalpayment
"""
rank = sqldf(rank, locals())

In [30]:
rank.head()

,client_id,entity_type,total_payment,rank
0,591,Australian Private Company,242349.96,1
1,1269,Australian Private Company,217316.74,2
2,1259,Australian Private Company,200541.98,3
3,1267,Australian Private Company,162987.53,4
4,298,Australian Private Company,108658.32,5


In [17]:
totalpayment.head()

,client_id,entity_type,total_payment
0,1,Other Partnership,2899.98
1,2,Australian Private Company,25950.03
2,3,Australian Private Company,1159.98
3,4,Australian Private Company,2319.96
4,6,Australian Private Company,12566.61


In [31]:
#Get the top 20 clients for 2018 sorted by total payment
top20 = """
    SELECT client_id,entity_type,total_payment,rank
    FROM rank
    WHERE rank <= 20
    ORDER BY total_payment DESC
"""
top20 = sqldf(top20, locals())


In [24]:
top20

,client_id,entity_type,total_payment,rank
0,591,Australian Private Company,242349.96,1
1,1269,Australian Private Company,217316.74,2
2,1259,Australian Private Company,200541.98,3
3,1195,Discretionary Investment Trust,172656.59,1
4,1267,Australian Private Company,162987.53,4
...,...,...,...,...
91,965,Australian Public Company,3383.31,4
92,42,Australian Public Company,2899.98,5
93,1,Other Partnership,2899.98,5
94,75,Other Partnership,941.66,6


In [32]:
top20.to_csv('D://BA//test DA//New folder (2)//top20.csv',index=False)